In [1]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
import augur

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            '', '', '',\
            '', ''
        )
db = s.create_engine(DB_STR)

In [3]:
# Url for the git repo to analyze
#repo_url = 'https://github.com/chaoss/grimoirelab-perceval'
repo_url = 'CTC'
# Directory for letting Perceval clone the git repo
#repo_dir = 'CTC'
token = ''
own = 'nodejs'
# ElasticSearch instance (url)
#repo = 'grimoirelab-perceval'

# Create the 'commits' index in ElasticSearch
# Create a Git object, pointing to repo_url, using repo_dir for cloning
repo = GitHub(owner=own,repository=repo_url,api_token=token)

In [5]:
#32
#date = datetime.datetime(2017, 8, 7, 0, 0, 0,
#                                     tzinfo=dateutil.tz.tzutc())
k=0
#for issues in repo.fetch(from_date=date):
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","issue_number","timestamp",\
          "issue_id","user","body"
df = pd.DataFrame(columns=columns1)
item = 1
df.to_sql(name="github_issues", con=db,\
          if_exists='replace',index=False,
           dtype={'augurmsgID': s.types.Integer,
                  'backend_name': s.types.VARCHAR(length=300),
                  'repo_link': s.types.VARCHAR(length=300),
                  'owner': s.types.VARCHAR(length=300),
                  'repo': s.types.VARCHAR(length=300),
                  'subject': s.types.VARCHAR(length=300),
                  'status': s.types.VARCHAR(length=10),
                  'category': s.types.VARCHAR(length=10),
                  'issue_number': s.types.Integer,
                  'timestamp': s.types.Integer,
                  'issue_id': s.types.Integer,
                  'user': s.types.VARCHAR(length=100),
                  'body':s.types.TEXT              
           })
for issues in repo.fetch():
    # Create the object (dictionary) to upload to ElasticSearch
    # Create the object (dictionary) to upload to ElasticSearch
    #if(i==90):
    if 'pull_request' in issues['data']:
        #print(issues['data']['number'])
        continue
    else:
        #if(issues['data']['state'] == 'closed' and issues['data']\
        #   ['number'] == 165): 
        text = issues['data']['body']
        user = issues['data']['user']['login']
        num = issues['data']['number']
        id_num = issues['data']['id']
        store = [item,issues['backend_name'],issues['tag'],own,repo_url,\
                 issues['data']['title'],issues['data']['state'],\
                 issues['category'],num,issues['timestamp'],\
                 id_num,user,text]
        #print(store[:])
        #print(store[6])
        #print("Issue number: ",issues['data']['number'])
        #if(k > 1):
            #k+=1
            #continue
        '''print("Backend_name: ", issues['backend_name'])
        print("Category: ", issues['category'])
        print("Subject: ", issues['data']['title'])
        print("State: ", issues['data']['state'])
        print("First body:",issues['data']['body'])
        print("User: ", issues['data']['user']['login'])
        print("\n\n\n")'''
        df_user = pd.DataFrame([store],columns=columns1)
        df = df.append(df_user)
        item+=1
        for i in range(len(issues['data']['comments_data'])):
            temp=0
            store[0] = item
            store[9] = issues['data']['id']
            store[10] = issues['data']['number']
            store[11] = issues['data']['comments_data'][i]\
            ['user_data']['login']
            store[12] = issues['data']['comments_data'][i]['body']
            df_user = pd.DataFrame([store],columns=columns1)
            df = df.append(df_user)
            #print(issues['data']['comments_data'][i]\
            #      ['user_data']['login'])
            #print(issues['data']['comments_data'][i]['body'])
            item+=1
          
    if(df.shape[0] > 1000):
        df.to_sql(name='github_issues', con=db,\
                  if_exists='append',index=False)
        df = pd.DataFrame(columns=columns1)
        print("Broken")
        #break
    #print(issues['data'][''])
    if(k%50 == 0):
        print(k)
    k+=1
    #if(k == 9):
    #    break
#print(df)
if(df.shape[0] < 1000):
    df.to_sql(name='github_issues', con=db,\
                  if_exists='append',index=False)
    df = pd.DataFrame(columns=columns1)
    print("Broken")
print("Total Number of issues: ", k)
print(df.shape[0])

2018-07-19 09:57:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/rvagg
2018-07-19 09:57:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/vkurchatkin
2018-07-19 09:57:19 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/benjamingr
2018-07-19 09:57:19 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/Trott
2018-07-19 09:57:19 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/joshgav
2018-07-19 09:57:20 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/Fishrock123


0


2018-07-19 09:57:21 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/jasnell
2018-07-19 09:57:21 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/addaleax
2018-07-19 09:57:22 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/ChALkeR
2018-07-19 09:57:23 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/bnoordhuis
2018-07-19 09:57:23 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/mhdawson
2018-07-19 09:57:24 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/williamkapke
2018-07-19 09:57:25 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/evanlucas
2018-07-19 09:57:25 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/MylesBorins
2018-07-19 09:57:26 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/of

2018-07-19 09:58:16 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/i5ting
2018-07-19 09:58:17 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/pesho
2018-07-19 09:58:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/thekemkid
2018-07-19 09:58:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/hxoht
2018-07-19 09:58:19 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/lamanabie
2018-07-19 09:58:19 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/CrabDude
2018-07-19 09:58:20 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/saschanaz
2018-07-19 09:58:20 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/leodutra
2018-07-19 09:58:20 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/DomVinyard
2018

50


...

2018-07-19 10:00:26 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/a0viedo
2018-07-19 10:00:28 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/fhemberger
2018-07-19 10:00:28 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/santigimeno
2018-07-19 10:00:29 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/sotayamashita
2018-07-19 10:00:31 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/danbev
2018-07-19 10:00:34 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/Eli-Goldberg
2018-07-19 10:00:35 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/DasWolke
2018-07-19 10:00:35 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/JamesTheHacker
2018-07-19 10:00:37 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com

2018-07-19 10:01:37 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/13241491189
2018-07-19 10:01:38 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/citycide
2018-07-19 10:01:38 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/curiousdannii
2018-07-19 10:01:39 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/fabm22
2018-07-19 10:01:40 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/robbiespeed
2018-07-19 10:01:40 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/tilgovi
2018-07-19 10:01:40 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/hiroppy
2018-07-19 10:01:41 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/sxa555
2018-07-19 10:01:41 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/Qix-


100


2018-07-19 10:02:06 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/srl295
2018-07-19 10:02:08 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/derekm
2018-07-19 10:02:11 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/jamiebuilds
2018-07-19 10:02:13 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/matthewloring
2018-07-19 10:02:15 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/hzoo
2018-07-19 10:02:17 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/Ks89
2018-07-19 10:02:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/leo
2018-07-19 10:02:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/halton
2018-07-19 10:02:18 keanu-Inspiron-5567 root[8675] INFO Getting info for https://api.github.com/users/stevenvachon
2018-07-

Broken
Total Number of issues:  126
0
